# Conversational Threads

Many LLM applications have a chatbot-like interface in which the user and the LLM application engage in a multi-turn conversation. In order to track these conversations, you can use the Threads feature in LangSmith.

This is relevant to our RAG application, which should maintain context from prior conversations with users.

### Setup

In [1]:
# You can set them inline
import os
os.environ["OPENAI_API_KEY"] = ""
os.environ["LANGSMITH_API_KEY"] = ""
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "langsmith-academy"  # If you don't set this, traces will go to the Default project

In [2]:
# Or you can use a .env file
from dotenv import load_dotenv
load_dotenv(dotenv_path=r".env", override=True)

True

### Group traces into threads


A Thread is a sequence of traces representing a single conversation. Each response is represented as its own trace, but these traces are linked together by being part of the same thread.

To associate traces together, you need to pass in a special metadata key where the value is the unique identifier for that thread.

The key value is the unique identifier for that conversation. The key name should be one of:

- session_id
- thread_id
- conversation_id.

The value should be a UUID.

In [3]:
import uuid
thread_id = uuid.uuid4()

In [4]:
from langsmith import traceable
from openai import OpenAI
from typing import List
import nest_asyncio
from utils import get_vector_db_retriever

openai_client = OpenAI()
nest_asyncio.apply()
retriever = get_vector_db_retriever()

@traceable(run_type="chain")
def retrieve_documents(question: str):
    return retriever.invoke(question)

@traceable(run_type="chain")
def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    rag_system_prompt = """You are an assistant for question-answering tasks. 
    Use the following pieces of retrieved context to answer the latest question in the conversation. 
    If you don't know the answer, just say that you don't know. 
    Use three sentences maximum and keep the answer concise.
    """
    messages = [
        {
            "role": "system",
            "content": rag_system_prompt
        },
        {
            "role": "user",
            "content": f"Context: {formatted_docs} \n\n Question: {question}"
        }
    ]
    return call_openai(messages)

@traceable(run_type="llm")
def call_openai(
    messages: List[dict], model: str = "gpt-4o-mini", temperature: float = 0.0
) -> str:
    return openai_client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )

@traceable(run_type="chain")
def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    return response.choices[0].message.content


USER_AGENT environment variable not set, consider setting it to identify your requests.


### Now let's run our application twice with this thread_id

In [5]:
question = "How do I add metadata to a Trace?"
ai_answer = langsmith_rag(question, langsmith_extra={"metadata": {"thread_id": thread_id}})
print(ai_answer)

/home/arnav/miniconda3/envs/llm/lib/python3.13/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
Failed to multipart ingest runs: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Unauthorized"}\n')trace=0471bdad-db2a-4beb-bbd7-2f287716d641,id=0471bdad-db2a-4beb-bbd7-2f287716d641; trace=0471bdad-db2a-4beb-bbd7-2f287716d641,id=24e49215-fb0e-4b98-9868-ef065596b996; trace=0471bdad-db2a-4beb-bbd7-2f287716d641,id=5ad0b753-58cf-4a81-a236-aff9792b8a93
Failed to send compressed multipart ingest: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Unauthorized"}\n')tra

To add metadata to a trace in LangSmith, you need to include a dictionary of key-value pairs when creating the trace. This metadata can store additional information about the trace, such as the environment or user details. Ensure that the same metadata is also attached to the corresponding run for better tracking.


Failed to send compressed multipart ingest: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Unauthorized"}\n')trace=0471bdad-db2a-4beb-bbd7-2f287716d641,id=b92ae3e5-cff5-4c7d-8c8d-ad9dfafc92d6; trace=0471bdad-db2a-4beb-bbd7-2f287716d641,id=c3894e8d-3c5f-4451-86e8-b02051464b2c; trace=0471bdad-db2a-4beb-bbd7-2f287716d641,id=0471bdad-db2a-4beb-bbd7-2f287716d641; trace=73b66cad-ac04-4cec-a6ae-c176f9677c8f,id=73b66cad-ac04-4cec-a6ae-c176f9677c8f; trace=73b66cad-ac04-4cec-a6ae-c176f9677c8f,id=ada2a2f5-49c1-4893-ae96-ac7119635eff; trace=73b66cad-ac04-4cec-a6ae-c176f9677c8f,id=a5236956-4e03-486d-a1a7-cc253dea140f; trace=73b66cad-ac04-4cec-a6ae-c176f9677c8f,id=a5236956-4e03-486d-a1a7-cc253dea140f; trace=73b66cad-ac04-4cec-a6ae-c176f9677c8f,id=ada2a2f5-49c1-4893-ae96-ac7119635eff; trace=73b66cad-ac04-4cec-a6ae-c176f9677c8f,id

In [6]:
question = "How can I add tags to a Trace?"
ai_answer = langsmith_rag(question, langsmith_extra={"metadata": {"thread_id": thread_id}})
print(ai_answer)

You can add tags to a trace in LangSmith by sending arbitrary metadata and tags along with the trace. Tags are strings used to categorize or label the trace, while metadata is a dictionary of key-value pairs for additional information. For detailed instructions, refer to the documentation on adding metadata and tags to traces.


In [7]:
question = "What's the difference between @traceable and wrap_openai?"
ai_answer = langsmith_rag(question, langsmith_extra={"metadata": {"thread_id": thread_id}})
print(ai_answer)

The `@traceable` decorator is used to mark specific functions for tracing, allowing you to log traces for those functions, while `wrap_openai` is a method that wraps the OpenAI client to automatically log traces without needing to modify the function code. Both can be used together in the same application, but `wrap_openai` simplifies the tracing process for OpenAI API calls. Essentially, `wrap_openai` provides a more seamless integration for tracing OpenAI interactions.


In [8]:
# Question 8: Advanced features
question = "How do I use RunTree for custom tracing hierarchies?"
ai_answer = langsmith_rag(question, langsmith_extra={"metadata": {"thread_id": thread_id}})
print(ai_answer)

To use RunTree for custom tracing hierarchies, you need to implement your own tracing logic that organizes runs into a tree structure. Each run can be a node in the tree, allowing you to represent the relationships and sequence of operations in your application. Ensure that your implementation adheres to the expected input and output formats recognized by LangSmith for proper logging and visualization.
